# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---
## 2022_09_02

### Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening 
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    

### To Test
1. Find whether the opening or closing that is causing the most issue in loss years
1. Test other years to understand how well this setup works.
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction

---

## 2022_09_02

### Observations
1. Opening and closing need to happen faster. Cant close after ema match
1. Lema angle helps save positions against frequent flips but delays closing and causes losses
1. Market is non-directional (Dec 2021 is a good example) Goes up and down, when algo tries to open and close its always too late
1. Need a faster method


### To Test
1. Should test plotting close as a line instead of candles. Perform match in close with wider window
1. Close chart needs to be analysed. This looks promising.
1. Go with simple rolling min max for close. Works better than resample min max merge

---


## 2022_09_13

### Observations
1. Lema crossing lema max and close using sema crossing lema max (This is the highest yielding 2020)
1. Need to analyze the gap between lema and lema max as an indicator for profit and loss


---

## Result by Years

| Year | Angle open | Angle close | Lema match open | Lema match close | Result type | pips  |
| ---- | ---------- | ----------- | --------------- | ---------------- | ----------- | ----- |
| 2022 | Yes        | No          | Yes             | Yes              | Profit      | 577   |
| 2021 | Yes        | No          | Yes             | Yes              | Loss        | \-596 |
| 2020 | Yes        | No          | Yes             | Yes              | Profit      | 1462  |
| 2019 | Yes        | No          | Yes             | Yes              | Loss        | 193   |
| 2018 | Yes        | No          | Yes             | Yes              | Profit      | 60    |
| 2017 | Yes        | No          | Yes             | Yes              | Profit      | \-1396|    

data                = get_date_list(data)
data                = read_data(data)

In [2]:
data["input_year"] = 2020
data['start_date'] = {'year':data["input_year"], 'month':1, 'date':1}
data['end_date']   = {'year':data["input_year"], 'month':12, 'date':31}
data["product"] = 'eurusd'
data["input_file_name"] = f'{data["product"]}_{data["input_year"]}.csv'

In [3]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 74387/74387 [00:18<00:00, 3946.46it/s]


pl    0.14534
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
64364 2020-11-12 12:10:00         NaN  sema_close -0.00279
65420 2020-11-18 04:10:00         NaN  sema_close  0.00484
65943 2020-11-19 23:45:00         NaN  sema_close -0.00311
66181 2020-11-20 19:35:00         NaN  sema_close -0.00281
66500 2020-11-23 22:10:00         NaN  sema_close -0.00435
67302 2020-11-26 17:00:00         NaN  sema_close  0.00082
67951 2020-11-30 23:05:00         NaN  sema_close -0.00063
69106 2020-12-06 23:20:00         NaN  sema_close  0.00827
69814 2020-12-09 10:20:00         NaN  sema_close -0.00201
70137 2020-12-10 13:15:00         NaN  sema_close -0.00450
70472 2020-12-11 17:10:00         NaN  sema_close -0.00334
71905 2020-12-18 16:35:00         NaN  sema_close  0.00790
72281 2020-12-21 23:55:00         NaN  sema_close -0.00193
73087 2020-12-24 19:05:00         NaN  sema_close -0.00347
74305 2020-12-31 15:10:00         NaN  sema_clo

100%|██████████████████████████████████████| 208/208 [00:00<00:00, 1173.35it/s]

Wall time: 2min 46s


In [4]:
data['df_ohlc'].groupby(['close_type'])['pl'].sum()

close_type
sema_close    0.14534
Name: pl, dtype: float64

In [5]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1     0.00075
2     0.01728
3     0.08514
4    -0.00293
5     0.01553
6     0.00758
7     0.02065
8    -0.01417
9     0.01023
10   -0.00654
11    0.00603
12    0.00579
Name: pl, dtype: float64
---------------------
month_val
8    -0.01417
10   -0.00654
4    -0.00293
1     0.00075
12    0.00579
11    0.00603
6     0.00758
9     0.01023
5     0.01553
2     0.01728
7     0.02065
3     0.08514
Name: pl, dtype: float64
---------------------
max : 3 : 0.08514000000000001
min : 8 : -0.014169999999999999
---------------------
monthly_avg : 0.0121


In [6]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 1]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [ ]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
chart_duration = 'month'
month = 12

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
# data['things_to_plot'] = ['candles','sema', 'close','lema', 'lema_match', 'positions', 'lema_change']
# data['things_to_plot'] = ['close', 'sema','lema', 'lema_change', 'lema_match', 'close_max']
# data['things_to_plot'] = ['close', 'tip','lema', 'close_max', 'close_min', 'lema_match']
# data['things_to_plot'] = ['close', 'lema','lema_match', 'slema', 'positions']
# data['things_to_plot'] = ['close', 'sema', 'slema','lema','lema_match', 'tip']
# data['things_to_plot'] = ['close', 'sema','lema','lema_match', 'match_max', 'match_min', 'unmatched']
# data['things_to_plot'] = ['close', 'sema','lema','lema_match', 'lema_max_min', 'positions']
data['things_to_plot'] = ['close','lema', 'sema', 'positions', 'lema_max_min']

data['plot_start']  = f"{data['input_year']}-{month}-1"

if chart_duration == 'month':
    if month == 12:
        data['plot_stop']   = f"{data['input_year']+1}-1-1"
    else:
        data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

elif chart_duration == 'year':        
    data['plot_start']  = f"{data['input_year']}-1-1"
    data['plot_stop']   = f"{data['input_year']+1}-1-1"


plot_graph(data)

generate_result_report(data)

In [7]:
temp = data['df_small'][['pl_type', 'pl','order_side','close', 'lema_max', 'lema_min', 'lema', 'sema']].dropna()

temp.loc[temp['order_side'] == 'long', 'lema_diff'] = temp['lema'] - temp['lema_max']
temp.loc[temp['order_side'] == 'short', 'lema_diff'] = temp['lema_min'] - temp['lema']

temp.loc[temp['order_side'] == 'long', 'sema_diff'] = temp['sema'] - temp['lema_max']
temp.loc[temp['order_side'] == 'short', 'sema_diff'] = temp['lema_min'] - temp['sema']

temp.loc[temp['order_side'] == 'long', 'close_diff'] = temp['close'] - temp['lema_max']
temp.loc[temp['order_side'] == 'short', 'close_diff'] = temp['lema_min'] - temp['close']

temp.head()

,pl_type,pl,order_side,close,lema_max,lema_min,lema,sema,lema_diff,sema_diff,close_diff
0,positive,0.00107,long,1.116655,1.116154,1.116153,1.116224,1.116382,0.000070,0.000228,0.000501
2,positive,0.00083,short,1.113055,1.119056,1.115736,1.115180,1.113692,0.000556,0.002044,0.002681
4,negative,-0.00112,long,1.112835,1.111290,1.111174,1.111525,1.112264,0.000235,0.000974,0.001545
6,negative,-0.00269,long,1.116030,1.113611,1.112762,1.113615,1.114993,0.000004,0.001382,0.002419
8,positive,0.00278,short,1.112610,1.115462,1.114195,1.113750,1.113197,0.000445,0.000998,0.001585


In [15]:
curve_type = 'sema_diff'

val_min = temp[curve_type].describe()['min'].round(5)
val_25 = temp[curve_type].describe()['25%'].round(5)
val_50 = temp[curve_type].describe()['50%'].round(5)
val_75 = temp[curve_type].describe()['75%'].round(5)
val_max = temp[curve_type].describe()['max'].round(5)

per_0_25 = temp.loc[(temp[curve_type] >= val_min) & (temp[curve_type] < val_25), 'pl'].sum().round(4)
per_25_50 = temp.loc[(temp[curve_type] >= val_25) & (temp[curve_type] < val_50), 'pl'].sum().round(4)
per_50_75 = temp.loc[(temp[curve_type] >= val_50) & (temp[curve_type] < val_75), 'pl'].sum().round(4)
per_75_100 = temp.loc[(temp[curve_type] >= val_75) & (temp[curve_type] <= val_max), 'pl'].sum().round(4)

per_0_25_count = temp.loc[(temp[curve_type] >= val_min) & (temp[curve_type] < val_25), 'pl'].count()
per_25_50_count = temp.loc[(temp[curve_type] >= val_25) & (temp[curve_type] < val_50), 'pl'].count()
per_50_75_count = temp.loc[(temp[curve_type] >= val_50) & (temp[curve_type] < val_75), 'pl'].count()
per_75_100_count = temp.loc[(temp[curve_type] >= val_75) & (temp[curve_type] <= val_max), 'pl'].count()

qrt = ['per_0_25' , 'per_25_50' , 'per_50_75' , 'per_75_100']
pl = [per_0_25 , per_25_50 , per_50_75 , per_75_100]
pl_count = [per_0_25_count , per_25_50_count , per_50_75_count , per_75_100_count]
qrt_start = [val_min, val_25, val_50, val_75]
qrt_stop = [val_25, val_50, val_75, val_max]

temp_report = pd.DataFrame(data = {'qrt':qrt, 'qrt_start':qrt_start, 'qrt_stop':qrt_stop, 'count':pl_count, 'pl':pl})
temp_report

,qrt,qrt_start,qrt_stop,count,pl
0,per_0_25,0.00023,0.00161,25,0.0153
1,per_25_50,0.00161,0.00207,25,0.0081
2,per_50_75,0.00207,0.00307,25,0.0590
3,per_75_100,0.00307,0.00987,26,0.0358
